### **EJEMPLO 4B**: TRABAJO SOBRE EL VIDEO

In [ ]:
import pandas as pd
import time 
import cv2 
import math

In [ ]:
# recorte vertical
x1 = 700
x2 = 1300
# recorte horizontal
y1 = 0
y2 = 800
# láser imaginario
yL = 400

# umbral
th = 190                      

# fijo el puerto (verificar el puerto de la camara ... en Windows ... averiguar)
port = 1

In [ ]:
cap = cv2.VideoCapture(port)            # si tengo una camara USB conectada
#cap = cv2.VideoCapture('cinta1.mov')   # si yo quiero analizar un video en vez de la camara
 
center_prev = []                # datos de los centros de masa anteriores
pasaron = 0                     # contador de pastillas



dir = False
dist_min = 30                    # d
area_min = 10000                  # area mínima para detección
area_max = 25000                  # area máxima para detección

start = time.time()
data = []                       # guarda los datos de tiempos

while cap.isOpened():
     
    ret, cuadro = cap.read()

    if not ret: break
    
    cuadro = cuadro[y1:y2,x1:x2,:]

    # convierto a escala de grises
    gray = cv2.cvtColor(cuadro, cv2.COLOR_BGR2GRAY)
 
    # hago la binarización 
    _, threshold = cv2.threshold(gray, th, 255, cv2.THRESH_BINARY)

    # encuentro los contornos de los objetos
    contours, _= cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    
    centers_curr = []           # información de los centros actual
    
    for c in contours:
        
        # calculo el area y la comparo con un area_min y area_max 
        if (cv2.contourArea(c) > area_min) and (cv2.contourArea(c) < area_max):

            # genero el bounding box
            (x, y, w, h) = cv2.boundingRect(c)

            # calculo el centro y dibujo centro y bounding box
            cx = int((2*x + w)/2)
            cy = int((2*y + h)/2)
            centers_curr.append((cx,cy))

            # dibuja rectangulo y punto central
            cv2.rectangle(cuadro, (x,y), (x+w, y+h), (0,255,0), 3)
            cv2.circle(cuadro, (cx, cy), 6, (0,0,255), -1)
    
    # verifico cuantos pasaron por la linea roja
    for p in centers_curr:
        for q in center_prev:

            # si un punto esta a menos de dist_min pixeles del anterior e Y2 > limit_line e Y1 =< limit_line, entonces cruzo la linea!
            distance = math.hypot(q[0]- p[0], q[1]-p[1]) 
            
            if ((dir==True) and distance<=dist_min and p[1]< yL and q[1]>= yL) or ((dir==False) and distance<=dist_min and p[1]> yL and q[1]<= yL):
                pasaron +=1
                data.append(time.time()-start)

    # copia las posiciones actuales
    center_prev = centers_curr.copy()

    # imprime el contador
    fontscale = 2
    line = 3
    cv2.putText(cuadro, str(pasaron), (5,yL - 12), cv2.FONT_HERSHEY_SIMPLEX , fontscale, (255, 255, 255), line, cv2.LINE_AA)

    # dibuja la linea roja
    cv2.line(cuadro, (0,yL), (640,yL), (0,0,255), 2)

    # muestra el cuadro
    cv2.imshow('CAMARA', cuadro ) #cuadro

    #cv2.waitKey(0)  # activar para ir cuadro a cuadro
    if cv2.waitKey(1) == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()